<a href="https://colab.research.google.com/github/AIDA2812/Sales_Forecasting_using_upgini_catboost/blob/main/Sales_Forecasting_using_upgini_catboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%pip install -Uq upgini

%pip install -Uq catboost


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.0/125.0 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.2/466.2 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.8/111.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [5]:
from os.path import exists
import pandas as pd
df_path = "train.csv.zip" if exists("train.csv.zip") else "https://github.com/upgini/upgini/raw/main/notebooks/train.csv.zip"
df = pd.read_csv(df_path)
df = df.sample(n=19_000, random_state=0)
df["store"] = df["store"].astype(str)
df["item"] = df["item"].astype(str)

df["date"] = pd.to_datetime(df["date"])
df.sort_values("date", inplace=True)
df.reset_index(inplace=True, drop=True)
df.head()



,date,store,item,sales
0,2013-01-01,7,5,5
1,2013-01-01,4,9,19
2,2013-01-01,1,33,37
3,2013-01-01,3,41,14
4,2013-01-01,5,24,26


In [6]:
train = df[df["date"] < "2017-01-01"]
test = df[df["date"] >= "2017-01-01"]

In [7]:
train_features = train.drop(columns=["sales"])
train_target = train["sales"]
test_features = test.drop(columns=["sales"])
test_target = test["sales"]


In [8]:
from upgini import FeaturesEnricher, SearchKey
from upgini.metadata import CVType

enricher = FeaturesEnricher(
    search_keys={
        "date": SearchKey.DATE,
    },
    cv = CVType.time_series
)
enricher.fit(train_features,
             train_target,
             eval_set=[(test_features, test_target)]
             )

[============================================================] 100% Finished

<IPython.core.display.Javascript object>

WARNING #1: Search started with DATE search key only
Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IP to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history


Detected task type: ModelTaskType.REGRESSION. Reason: date search key is present, treating as regression
You can set task type manually with argument `model_task_type` of FeaturesEnricher constructor if task type detected incorrectly

WARNING #2: Your training sample is unstable in number of rows per date. It is recommended to redesign the training sample



<IPython.core.display.Javascript object>

Column name,Status,Errors
target,All valid,-
date,All valid,-


...

f_autofe_groupbythenmean_d5f4f809f4,8.4902,100.0000,"0.0981, 0.0687, 0.0905","Upgini,Training dataset","AutoFE: feature from Markets data, grouped by feature from training dataset",Daily
f_economic_date_cbpol_umap_6_aa0352de,7.3931,100.0000,"1.0587, 6.9596, 1.0408",Upgini,World economic indicators,Daily
f_autofe_groupbythenmedian_64b28257d7,6.5439,100.0000,"1.0005, 1.0012, 1.0004","Upgini,Training dataset","AutoFE: feature from Markets data, grouped by feature from training dataset",Daily
f_autofe_groupbythenstd_9a84d2fd29,5.5322,100.0000,"0.7213, 0.7161, 0.6952","Upgini,Training dataset","AutoFE: feature from Calendar data, grouped by feature from training dataset",Daily
f_autofe_groupbythenrank_d595da62f7,3.6036,100.0000,"0.3398, 0.5069, 0.2105","Upgini,Training dataset","AutoFE: feature from Calendar data, grouped by feature from training dataset",Daily
f_events_date_year_cos1_9014a856,2.3165,100.0000,"0.3253, -0.263, -0.3496",Upgini,Calendar data,Daily
f_financial_date_crude_oil_7d_to_1y_c3e0ad17,2.2541,100.0000,"1.0001, 1.0769, 1.0154",Upgini,Markets data,Daily
f_autofe_roll_3d_median_8e12f1e1e9,1.4845,100.0000,"0.309, -0.263, -0.3496","Training dataset,Upgini","AutoFE: features from Training dataset,Calendar data",Daily
f_autofe_roll_3d_min_1a2ad6d8bb,1.4531,100.0000,"0.3253, -0.2795, -0.3817","Training dataset,Upgini","AutoFE: features from Training dataset,Calendar data",Daily
item,1.0343,100.0000,"40, 30, 18",,,
f_autofe_groupbythenrank_110e5b8df1,0.8715,100.0000,"0.0758, 0.2147, 0.7818","Upgini,Training dataset","AutoFE: feature from Calendar data, grouped by feature from training dataset",Daily


"Upgini,Training dataset","AutoFE: feature from Markets data, grouped by feature from training dataset",15.0341,2
"Upgini,Training dataset","AutoFE: feature from Calendar data, grouped by feature from training dataset",10.0739,4
Upgini,World economic indicators,7.3931,1
Upgini,Calendar data,3.3263,4
"Training dataset,Upgini","AutoFE: features from Training dataset,Calendar data",2.9376,2
Upgini,Markets data,2.2541,1
Training dataset,AutoFE: features from Training dataset,1.5778,3


"Markets data, grouped by feature from training dataset",f_autofe_groupbythenmean_d5f4f809f4,f_financial_date_vix_gap_b64bd2b9,store_824d80,GroupByThenMean
"Markets data, grouped by feature from training dataset",f_autofe_groupbythenmedian_64b28257d7,f_financial_date_usd_eur_1d_to_7d_d8b89b5d,item_4a33ea,GroupByThenMedian
"Calendar data, grouped by feature from training dataset",f_autofe_groupbythenstd_9a84d2fd29,f_events_date_year_cos8_d2a83ca9,item_4a33ea,GroupByThenStd
"Calendar data, grouped by feature from training dataset",f_autofe_groupbythenrank_d595da62f7,f_events_date_week_cos3_7525fe31,store_824d80,GroupByThenRank
"Training dataset,Calendar data",f_autofe_roll_3d_median_8e12f1e1e9,f_events_date_year_cos1_9014a856,,roll_3d_median
"Training dataset,Calendar data",f_autofe_roll_3d_min_1a2ad6d8bb,f_events_date_year_cos1_9014a856,,roll_3d_min
"Calendar data, grouped by feature from training dataset",f_autofe_groupbythenrank_110e5b8df1,f_events_date_week_cos1_f6a8c1fc,store_824d80,GroupByThenRank
Training dataset,f_autofe_lag_28d_aaadd46bbb,target,,lag_28d
Training dataset,f_autofe_lag_7d_4a1d9c9dff,target,,lag_7d
Training dataset,f_autofe_roll_3d_median_97804b26e6,target,,roll_3d_median
"Calendar data, grouped by feature from training dataset",f_autofe_groupbythenrank_3729e8053b,f_events_date_week_cos1_f6a8c1fc,item_4a33ea,GroupByThenRank


We detected 48 outliers in your sample.
Examples of outliers with maximum value of target:
33    205
17    196
12    187
Name: target, dtype: int64
Outliers will be excluded during the metrics calculation.
Calculating accuracy uplift after enrichment...
y distributions from the training sample and eval_set differ according to the Kolmogorov-Smirnov test,
which makes metrics between the train and eval_set incomparable.


Train,9418,53.3352,0.324 ± 0.109,0.217 ± 0.086,0.1070,33.0%
Eval 1,3764,58.5994,0.278 ± 0.009,0.195 ± 0.021,0.0820,29.7%


The error `NameError: name 'enricher' is not defined` indicates that the variable `enricher` was not created in the current session. This is likely because the cell where `enricher` is defined and fit (`foJr0ZXsqvSR`) has not been executed. To fix this, we need to run that cell first.

In [ ]:

from catboost import CatBoostRegressor
from catboost.utils import eval_metric

model = CatBoostRegressor(verbose=False, allow_writing_files=False, random_state=0)
enricher.calculate_metrics(
    train_features, train_target,
    eval_set=[(test_features, test_target)],
    estimator=model,
    scoring= "mean_absolute_percentage_error"
)

Calculating accuracy uplift after enrichment...
y distributions from the training sample and eval_set differ according to the Kolmogorov-Smirnov test,
which makes metrics between the train and eval_set incomparable.


In [ ]:
enriched_train_features = enricher.fit_transform(train_features, train_target, keep_input=True)
enriched_test_features = enricher.transform(test_features, keep_input=True)


enriched_train_features.head()

In [ ]:
model.fit(train_features, train_target)
preds = model.predict(test_features)
eval_metric(test_target.values, preds, "SMAPE")

In [ ]:
model.fit(enriched_train_features, train_target)
enriched_preds = model.predict(enriched_test_features)
mape = eval_metric(test_target.values, enriched_preds, "SMAPE")

In [ ]:
# 1. Install Libraries
# - Install and update Upgini (for feature enrichment)
# - Install and update CatBoost (for sales prediction modeling)

# 2. Read & Prepare Data
# - Load dataset from local file or GitHub
# - Take a sample of 19,000 rows to reduce size
# - Convert 'store' and 'item' to string
# - Convert 'date' to datetime format
# - Sort by date and reset index
# - Display first 5 rows

# 3. Split Data (Train/Test)
# - Train set = rows before 2017
# - Test set = rows from 2017 onwards
# - Separate features (X) from target 'sales' (y)

# 4. Feature Enrichment with Upgini
# - Define 'date' as the time key
# - Use time series cross-validation
# - Fit the enricher on training data and validate on test data
# - Goal: add external features (e.g., weather, holidays) to improve accuracy

# 5. Define CatBoost Model
# - Create CatBoost Regressor for numerical prediction
# - Set verbose=False to hide logs
# - Set random_state=0 for reproducibility

# 6. Evaluate Upgini + CatBoost
# - Measure model performance with MAPE (Mean Absolute Percentage Error)
# - Purpose: compare accuracy before vs after enrichment

# 7. Train Model Without Enrichment
# - Train CatBoost on original train set
# - Predict sales on test set
# - Evaluate with SMAPE (Symmetric Mean Absolute Percentage Error)

# 8. Train Model With Enriched Features
# - Train CatBoost on enriched train set
# - Predict on enriched test set
# - Re-evaluate with SMAPE to check performance improvement
